In [1]:
from moviepy.editor import VideoFileClip
import scipy.stats
import pandas as pd
import glob
import json
import librosa
import numpy as np
import soundfile as sf
import io

In [2]:
# video_clip = VideoFileClip("train_sample_videos/aagfhgtpmv.mp4")
# audio = video_clip.audio

# # Step 2: Convert the audio into an array of samples
# fps = audio.fps  # frames per second (sampling rate)
# audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()

# # Step 3: Use an in-memory buffer to store the audio as a .wav file
# buffer = io.BytesIO()
# sf.write(buffer, audio_samples, fps, format='wav')  # Store audio in buffer as .wav
# buffer.seek(0)  # Reset buffer to the beginning

# # Step 4: Load audio from buffer using librosa
# x, sr = librosa.load(buffer, sr=None)

Statistical Features  
A first easy step is to compute the mean, standard deviation, minimum, maximum, median and quartiles of the frequencies of each signal. This can be done using Numpy and it always brings value to our feature extraction.

In [3]:
# freqs = np.fft.fftfreq(x.size)

# def describe_freq(freqs):
#     mean = np.mean(freqs)
#     std = np.std(freqs) 
#     maxv = np.amax(freqs) 
#     minv = np.amin(freqs) 
#     median = np.median(freqs)
#     skew = scipy.stats.skew(freqs)
#     kurt = scipy.stats.kurtosis(freqs)
#     q1 = np.quantile(freqs, 0.25)
#     q3 = np.quantile(freqs, 0.75)
#     mode = scipy.stats.mode(freqs)[0][0]
#     iqr = scipy.stats.iqr(freqs)
    
#     return [mean, std, maxv, minv, median, skew, kurt, q1, q3, mode, iqr]

In [4]:
# mfcc = list(np.mean(librosa.feature.mfcc(y=x, sr=sr), axis=1))
# print(len(mfcc))

In [5]:
def get_features(x, sr):
    rmse = np.mean(librosa.feature.rms(y=x)[0])
    zcr = np.mean(librosa.feature.zero_crossing_rate(x)[0])
    tempo = librosa.beat.tempo(y=x, sr=sr)[0]
    mfcc = list(np.mean(librosa.feature.mfcc(y=x, sr=sr), axis=1))
    spec_cen = np.mean(librosa.feature.spectral_centroid(y=x, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=x, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=x, sr=sr))
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=x))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=x, sr=sr))
    features = [rmse, zcr, tempo, spec_cen, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff]
    return features + mfcc

In [6]:
column_names = ['FileName', 'rmse', 'zcr', 'tempo', 'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_flatness', 'spectral_rolloff'] + ['mfcc' + str(i) for i in range(1, 21)] + ['label']
file_names = glob.glob("train_sample_videos/*.mp4")
df = pd.DataFrame(columns=column_names)

In [7]:
# file = "train_sample_videos/aagfhgtpmv.mp4"

In [8]:
# clean_file = file.split("/")[-1]
# video_clip = VideoFileClip(file)
# audio = video_clip.audio
# fps = audio.fps
# audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()
# buffer = io.BytesIO()
# sf.write(buffer, audio_samples, fps, format='wav')
# buffer.seek(0)
# x, sr = librosa.load(buffer, sr=None)
# label = json.load(open("train_sample_videos/metadata.json"))[clean_file]['label']
# new_row = pd.DataFrame([[clean_file] + get_features(x, sr) + [label]], columns=column_names)
# df = pd.concat([df, new_row], ignore_index=True)

In [9]:
# df

In [10]:
for file in file_names:
    clean_file = file.split("/")[-1]
    video_clip = VideoFileClip(file)
    audio = video_clip.audio
    fps = audio.fps
    audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()
    buffer = io.BytesIO()
    sf.write(buffer, audio_samples, fps, format='wav')
    buffer.seek(0)
    x, sr = librosa.load(buffer, sr=None)
    label = json.load(open("train_sample_videos/metadata.json"))[clean_file]['label']
    new_row = pd.DataFrame([[clean_file] + get_features(x, sr) + [label]], columns=column_names)
    df = pd.concat([df, new_row], ignore_index=True)

In [11]:
df

,FileName,rmse,zcr,tempo,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,spectral_rolloff,mfcc1,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,bbvgxeczei.mp4,0.008028,0.038316,114.843750,3044.967492,5054.150023,16.414213,0.005748,5682.982621,-490.074249,...,4.405777,-4.854592,-7.056202,-2.636595,-9.831662,-7.514642,-8.755273,-1.896833,-2.603562,FAKE
1,agqphdxmwt.mp4,0.005214,0.041688,123.046875,3522.410712,5273.395038,15.003604,0.007969,7564.151674,-544.736633,...,12.557711,-6.926795,-1.758291,-2.297427,-6.438809,-0.577922,-3.541346,-1.711840,-0.793243,FAKE
2,cglxirfaey.mp4,0.000189,0.073073,120.185320,5057.993566,5686.422947,13.394466,0.038380,12171.683589,-809.702148,...,7.754112,-0.916289,3.209106,-1.237075,1.662170,-4.138675,3.358046,-2.736235,0.954534,FAKE
3,ekhacizpah.mp4,0.032205,0.023700,126.048018,2369.255016,4806.154161,16.368527,0.001560,3229.456789,-340.318542,...,2.123348,-3.004771,-2.435924,5.287930,-9.668341,0.485537,-0.291376,6.767036,-2.478494,FAKE
4,ahbweevwpv.mp4,0.031066,0.019815,117.453835,2084.400204,4570.329358,15.247853,0.001029,2592.238562,-361.499481,...,5.199107,3.196825,-3.196581,0.345530,-10.903251,2.098206,-4.634393,4.790202,-2.696295,FAKE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,btjlfpzbdu.mp4,0.062160,0.011180,126.048018,1652.398101,4098.890492,16.571897,0.000542,2069.194938,-351.217255,...,7.138178,4.056450,-0.416696,4.311500,-4.964599,-0.078893,2.780434,7.019897,2.743258,FAKE
396,duzuusuajr.mp4,0.010226,0.033884,123.046875,2799.805994,5002.288248,15.869884,0.003673,4779.298798,-458.709625,...,0.454649,-5.337661,-7.836361,-2.688951,-10.671743,-6.645610,-8.645223,-2.095698,-3.208719,FAKE
397,dxuliowugt.mp4,0.008680,0.029999,123.046875,2686.293526,4930.656757,16.068878,0.004149,4383.434486,-489.672760,...,4.150144,-3.787300,-6.313182,-1.511123,-10.802405,-3.185559,-8.830258,-2.280676,-5.673003,FAKE
398,dozyddhild.mp4,0.037665,0.041988,117.453835,3092.296261,5213.102108,18.349124,0.004358,4943.185550,-337.551086,...,-6.772953,-7.396572,-14.746246,-3.176184,-2.532345,3.301229,-19.290184,-1.723458,-3.074266,FAKE


In [14]:
df.to_csv("features.csv", index=False)